shuffle the order of the elements in the tensor

In [1]:
import tensorflow as tf

In [2]:
#shuffle a tensor (Valuable for when you want to shuffle your data so the inherint order doesn't effect learning)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])
not_shuffled.ndim

2

In [3]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]])>

In [4]:
# shuffling the tensorflow (not-shuffled)
tf.random.shuffle(not_shuffled)# now the order will change

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 2,  5],
       [ 3,  4]])>

In [5]:
# shuffling the tensorflow (not-shuffled)
tf.random.shuffle(not_shuffled)# now the order will change again

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [10,  7],
       [ 3,  4]])>

In [6]:
# shuffling the tensorflow (not-shuffled)
tf.random.shuffle(not_shuffled, seed = 42)# now the order will change

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [ 3,  4],
       [10,  7]])>

In [7]:
tf.random.set_seed(42)#here set_seed is global level seed.second step has operation-level seed
tf.random.shuffle(not_shuffled, seed =42)#now here how many times we run it is not shuffled bcoz it has global and operatonal-level seeds

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]])>

Exercise : Read through tensorflow documentation on random seed generation https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensors and shuffle them

Operations that rely on a random seed actually derive it from two seeds: the global and operation-level seeds. This sets the global seed.

Its interactions with operation-level seeds is as follows:

If neither the global seed nor the operation seed is set: A randomly picked seed is used for this op.
If the global seed is set, but the operation seed is not: The system deterministically picks an operation seed in conjunction with the global seed so that it gets a unique random sequence. Within the same version of tensorflow and user code, this sequence is deterministic. However across different versions, this sequence might change. If the code depends on particular seeds to work, specify both global and operation-level seeds explicitly.
If the operation seed is set, but the global seed is not set: A default global seed and the specified operation seed are used to determine the random sequence.
If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence.
To illustrate the user-visible effects, consider these examples:

If neither the global seed nor the operation seed is set, we get different results for every call to the random op and every re-run of the program:


print(tf.random.uniform([1]))  # generates 'A1'
print(tf.random.uniform([1]))  # generates 'A2'
(now close the program and run it again)


print(tf.random.uniform([1]))  # generates 'A3'
print(tf.random.uniform([1]))  # generates 'A4'


If the global seed is set but the operation seed is not set, we get different results for every call to the random op, but the same sequence for every re-run of the program:


tf.random.set_seed(1234)
print(tf.random.uniform([1]))  # generates 'A1'
print(tf.random.uniform([1]))  # generates 'A2'
(now close the program and run it again)


tf.random.set_seed(1234)
print(tf.random.uniform([1]))  # generates 'A1'
print(tf.random.uniform([1]))  # generates 'A2'


The reason we get 'A2' instead 'A1' on the second call of tf.random.uniform above is because the second call uses a different operation seed.

Note that tf.function acts like a re-run of a program in this case. When the global seed is set but operation seeds are not set, the sequence of random numbers are the same for each tf.function. For example:


tf.random.set_seed(1234)

@tf.function
def f():
  a = tf.random.uniform([1])
  b = tf.random.uniform([1])
  return a, b

@tf.function
def g():
  a = tf.random.uniform([1])
  b = tf.random.uniform([1])
  return a, b

print(f())  # prints '(A1, A2)'
print(g())  # prints '(A1, A2)'


If the operation seed is set, we get different results for every call to the random op, but the same sequence for every re-run of the program:


print(tf.random.uniform([1], seed=1))  # generates 'A1'
print(tf.random.uniform([1], seed=1))  # generates 'A2'
(now close the program and run it again)


print(tf.random.uniform([1], seed=1))  # generates 'A1'
print(tf.random.uniform([1], seed=1))  # generates 'A2'


The reason we get 'A2' instead 'A1' on the second call of tf.random.uniform above is because the same tf.random.uniform kernel (i.e. internal representation) is used by TensorFlow for all calls of it with the same arguments, and the kernel maintains an internal counter which is incremented every time it is executed, generating different results.


Calling tf.random.set_seed will reset any such counters:


tf.random.set_seed(1234)
print(tf.random.uniform([1], seed=1))  # generates 'A1'
print(tf.random.uniform([1], seed=1))  # generates 'A2'
tf.random.set_seed(1234)
print(tf.random.uniform([1], seed=1))  # generates 'A1'
print(tf.random.uniform([1], seed=1))  # generates 'A2'

When multiple identical random ops are wrapped in a tf.function, their behaviors change because the ops no long share the same counter. For example:


@tf.function
def foo():
  a = tf.random.uniform([1], seed=1)
  b = tf.random.uniform([1], seed=1)
  return a, b
print(foo())  # prints '(A1, A1)'
print(foo())  # prints '(A2, A2)'

@tf.function
def bar():
  a = tf.random.uniform([1])
  b = tf.random.uniform([1])
  return a, b
print(bar())  # prints '(A1, A2)'
print(bar())  # prints '(A3, A4)'


The second call of foo returns '(A2, A2)' instead of '(A1, A1)' because tf.random.uniform maintains an internal counter. If you want foo to return '(A1, A1)' every time, use the stateless random ops such as tf.random.stateless_uniform. Also see tf.random.experimental.Generator for a new set of stateful random ops that use external variables to manage their states.

In [8]:
random_1 = tf.random.Generator.from_seed(7)
random_1 = random_1.normal(shape = (4,4))
tf.random.shuffle(random_1)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[-0.29604465, -0.21134205,  0.01063002,  1.5165398 ],
       [-1.3240396 ,  0.28785667, -0.8757901 , -0.08857018],
       [ 0.69211644,  0.84215707, -0.06378496,  0.92800784],
       [-0.6039789 , -0.1766927 ,  0.04221033,  0.29037967]],
      dtype=float32)>

In [9]:
tf.random.shuffle(random_1)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[-0.6039789 , -0.1766927 ,  0.04221033,  0.29037967],
       [-1.3240396 ,  0.28785667, -0.8757901 , -0.08857018],
       [-0.29604465, -0.21134205,  0.01063002,  1.5165398 ],
       [ 0.69211644,  0.84215707, -0.06378496,  0.92800784]],
      dtype=float32)>

In [10]:
tf.random.set_seed(1234)
print(tf.random.uniform([2,2])) 

tf.Tensor(
[[0.5380393  0.36461866]
 [0.5816301  0.24382842]], shape=(2, 2), dtype=float32)


In [11]:
tf.random.set_seed(1234)

tf.function
def f():
  a = tf.random.uniform([2,2])
  tf.random.shuffle(a)
  b = tf.random.uniform([1])
  return a, b

tf.function
def g():
  a = tf.random.uniform([2,2])
  tf.random.shuffle(a)
  b = tf.random.uniform([1])
  return a, b

print(f())  # prints '(A1, A2)'
print(g())  # prints '(A1, A2)'

(<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.5380393 , 0.36461866],
       [0.5816301 , 0.24382842]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.59750986], dtype=float32)>)
(<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.9099175 , 0.6676756 ],
       [0.19938636, 0.3284049 ]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.3780992], dtype=float32)>)


It looks like if we want our random_1 tensors to be in same order we've
got use the global level and as well as operation level random seed.

Rule 4: If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random seq

In [12]:
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(random_1, seed = 42) # operation level random seed

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[ 0.69211644,  0.84215707, -0.06378496,  0.92800784],
       [-0.6039789 , -0.1766927 ,  0.04221033,  0.29037967],
       [-1.3240396 ,  0.28785667, -0.8757901 , -0.08857018],
       [-0.29604465, -0.21134205,  0.01063002,  1.5165398 ]],
      dtype=float32)>

# Other ways to make tensors

https://www.tensorflow.org/api_docs/python/tf/ones

In [13]:
# create a tensor of all ones
tf.ones(shape = [10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [14]:
# create a tensor of all zero's 

tf.zeros(shape = (3,5))

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

### Turn Numpy arrays into tensors

The main difference between Numpy arrays and tensorflow tensors is that tensors can be run on a Gpu (much faster fornumerical computing)

tf.constant(
    value, dtype=None, shape=None, name='Const')


value:
A constant value (or list) of output type dtype.
dtype:
The type of the elements of the resulting tensor.
shape:
Optional dimensions of resulting tensor.
name:
Optional name for the tensor.


Returns:
A Constant Tensor.
Raises:
TypeError

if shape is incorrectly specified or unsupported.:
ValueError:
if called on a symbolic tensor.

In [15]:
# You can also convert numpy arrays to tensors

import numpy as np

numpy_A = np.arange(1, 25, dtype = np.int32) # create a numpy array b/w 1 to 25
numpy_A
#X = tf.constant(some_matrix) # capital for matrix or tensors (numpy_A)
#y = tf.constant(vector) # non capital for vectors (numpy_a)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [16]:
A = tf.constant(numpy_A)
A # here it was converted numpy array to tensors

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])>

In [17]:
A = tf.constant(numpy_A, shape = (2, 3, 4)) #(2 *3 *4) = 24
B = tf.constant(numpy_A)

In [18]:
A,  B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]])>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24])>)

In [19]:
A = tf.constant(numpy_A, shape = (4,6)) #(4 * 6) = 24
B = tf.constant(numpy_A)

A,  B

(<tf.Tensor: shape=(4, 6), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6],
        [ 7,  8,  9, 10, 11, 12],
        [13, 14, 15, 16, 17, 18],
        [19, 20, 21, 22, 23, 24]])>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24])>)

In [20]:
A.ndim

2

### Getting information from tensors

When dealing with tensors you probably want to be aware of the following:

* shape == The length (number of elements) of each of the dimensions of a tensor. == tensor.shape
* Rank == The number of tensor dimensions.A scalar has rank 0, a vector has rank 1, a matrix has rank 2 or 2 dimension, a tensor has rank n or n dimension.
  == tensor.ndim
* Axis or dimension == A particular dimension of a tensor == tensor[0], tensor[:, 1]
* size == The Total number of items in the tensors == tf.size(tensor)

In [21]:
# create a rank 4 tensor (4 dimensions)

rank_4_tensor = tf.zeros(shape=[2,3,4,5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [22]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [23]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [24]:
# Get various attribute of our tensor

print('Datatype of every element: ', rank_4_tensor.dtype)
print('Number of dimensions: ', rank_4_tensor.ndim)
print('Shape of tensor: ', rank_4_tensor.shape)
print('Elements along the 0 axis: ', rank_4_tensor.shape[0])
print('Elements along the last axis: ', rank_4_tensor.shape[-1])
print("Total number of elements in our tensor:", 
      tf.size(rank_4_tensor))
print("Total number of elements in our tensor:", 
      tf.size(rank_4_tensor).numpy())

Datatype of every element:  <dtype: 'float32'>
Number of dimensions:  4
Shape of tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  2
Elements along the last axis:  5
Total number of elements in our tensor: tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor: 120


### Indexing tensors

Tensors can be indexed just like python lists

In [25]:
# Get the first 2 elements of each dimension

some_list = [1, 2, 3, 4]
some_list[:2]

[1, 2]

In [26]:
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [27]:
# Get the first element from each dimension from each index except for the final one.

rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [28]:
# Get the first element from each dimension from each index except for second last one.

rank_4_tensor[:1, :1, :, :1]

<tf.Tensor: shape=(1, 1, 4, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         [0.]]]], dtype=float32)>

In [29]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[10, 7],
                            [3, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [30]:
rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]])>

In [31]:
 some_list, some_list[-1]

([1, 2, 3, 4], 4)

In [32]:
# Get the last element of each of row of our rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4])>

In [33]:
# add extra dimension to our rank_2_dimension
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]])>

In [34]:
# alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis = -1) # "-1" means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]])>

In [35]:
tf.expand_dims(rank_2_tensor, axis = 0) # 0 means adding extra dimension at front

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]])>

In [36]:
tf.expand_dims(rank_2_tensor, axis = 1) # 1 means adding extra dimension at the middle

<tf.Tensor: shape=(2, 1, 2), dtype=int32, numpy=
array([[[10,  7]],

       [[ 3,  4]]])>

In [37]:
rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]])>

### Manipulating tensor  (tensor operations)

Basic operations

In [38]:
# you can add values to the tensor using addition operator
tensor = tf.constant([[10, 7], [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]])>

In [39]:
# now original tensor will not change
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]])>

In [40]:
# only way to change 
tensor = tensor + 10
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]])>

In [41]:
# multiplication 
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[200, 170],
       [130, 140]])>

In [42]:
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]])>

In [43]:
# we can use the tensor built in function too
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[200, 170],
       [130, 140]])>

In [44]:
tensor# original tensor doesn't change

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]])>

Matrix Multiplication

In machine learning , matrix multiplication is one of the most common tensor operations.

There are two rules our tensors (or matrices) need to fulfil if we're going to matrix multiply them:
1. The inner dimensions must match
2. The resulting matrix has the shape of the outer dimensions.

In [45]:
# matrix multiplication in tensorflow

print(tensor)
tf.matmul(tensor,tensor)

tf.Tensor(
[[20 17]
 [13 14]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[621, 578],
       [442, 417]])>

In [46]:
# matrix multiplication with python operator "@"

tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[621, 578],
       [442, 417]])>

In [47]:
tensor.shape

TensorShape([2, 2])

In [48]:
# create a tensor (3, 2)
X = tf.constant([[1, 2],
                [3, 4],
                [5, 6]])

# create another (3, 2) tensor
y = tf.constant([[7, 8],
                [9, 10],
                [11, 12]])

X, y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]])>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]])>)

In [49]:
# Try to matrix multiply of same shape
# we will get error bcoz we have to satisfy the two rules

#There are two rules our tensors (or matrices) need to fulfil 
#if we're going to matrix multiply them:

# 1.The inner dimensions must match
# 2.The resulting matrix has the shape of the outer dimensions.
# means for first rule if 2 matrices have (x, y) and another 
# matrix have (a, b) size, for 1st rule in inner dim if y = a
#  This y = a is satisfies 1 st rule and for second rule x = b must
# be satisfied.


# tf.matmul(X, y) # you will get error here bcoz you are not following the above rules

In [50]:
y

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 7,  8],
       [ 9, 10],
       [11, 12]])>

In [51]:
# Let's change the shape of y
tf.reshape(y, shape = (2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]])>

In [52]:
X

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6]])>

In [53]:
X.shape, tf.reshape(y, shape = (2, 3)).shape # checking whether the
# two rules are satisfied or not.

(TensorShape([3, 2]), TensorShape([2, 3]))

In [54]:
# try to mmatrix multiply X by reshaped y

X @ tf.reshape(y, shape = (2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]])>

In [55]:
tf.matmul(X, tf.reshape(y, shape = (2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]])>

In [56]:
# what if we reshape X instead of y

tf.matmul(tf.reshape(X, shape = (2, 3)), y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]])>

In [57]:
tf.reshape(X, shape = (2, 3)).shape, y.shape

(TensorShape([2, 3]), TensorShape([3, 2]))

In [58]:
# can do the same with transpose
X, tf.transpose(X), tf.reshape(X, shape = (2, 3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]])>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]])>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]])>)

In [59]:
# Try matrix multiplication with transpose rather than reshape
tf.matmul(tf.transpose(X), y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]])>

**The dot product**

Matrix multiplication is also reffered to as the dot product.
You can perform matrix multiplication using:
* 'tf.matmul()'
* 'tf.tensordot()'

In [60]:
X, y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]])>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]])>)

In [61]:
# perform the dot product X and y (requires X or y to be transpose)
# tf.tensordot(): tensor contraction of a and b along specified axes and outer product.
tf.tensordot(tf.transpose(X), y, axes = 1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]])>

In [62]:
# perform matrix multiplication b/w  X and y (transposed)
tf.matmul(X, tf.transpose(y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]])>

In [63]:
# perform matrix multiplication b/w  X and y (reshaped)
tf.matmul(X, tf.reshape(y, shape = (2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]])>

In [64]:
# check the values of y, reshape y and transposed y
print('normal y: ', y)
print('reshaped y: ', tf.reshape(y, shape = (2, 3)))
print('transpose y: ', tf.transpose(y))

normal y:  tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32)
reshaped y:  tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32)
transpose y:  tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


In [65]:
tf.matmul(X, tf.transpose(y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]])>

Generally, when performing matrix multiplication on two tensors and one of the axes doesn't line up, you will transpose (rather than reshape) one of the tensors to get satisfy the matrix multiplication rules.

# Changing the datatype of a tensor

In [66]:
# create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B, B.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 tf.float32)

In [67]:
C = tf.constant([7, 10])
C.dtype

tf.int32

In [68]:
# change from float32 to float16 (called reduced precision)
D = tf.cast(B, dtype = tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [69]:
# change from int32 to float32
E = tf.cast(C, dtype = tf.float32)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>,
 tf.float32)

In [70]:
E_float16 = tf.cast(E, dtype = tf.float16)
E_float16, E_float16.shape

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([ 7., 10.], dtype=float16)>,
 TensorShape([2]))

# Aggregating tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [71]:
# Create a tensor
D = tf.constant([-7, -10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10])>

In [72]:
# Get the absolute values
tf.abs(D)# So the absolute values are basically take all the -ve numbers in one tensor and turn them into +ve numbers.

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10])>

Let's go through the following forms of aggregation:

* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor

In [73]:
# creating a random tensorwith values b/w 0 and 100 o size 50
E = tf.constant(np.random.randint(1, 100, size = 50))
E

<tf.Tensor: shape=(50,), dtype=int32, numpy=
array([89, 50, 74, 20, 92, 52,  7, 99, 19, 84, 69, 44, 29, 18, 60, 14, 38,
       21, 70, 68, 62, 21, 14, 79, 86, 97, 73, 31, 47, 65,  1, 84, 30, 69,
       59, 38, 15, 27, 91, 40, 53, 71, 17, 19,  6,  5, 28, 92, 85, 76])>

In [74]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [75]:
# Find the minimum 
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [76]:
# Find the maximum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int32, numpy=99>

In [77]:
# Find the mean
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int32, numpy=49>

In [78]:
# Find the sum of tensor
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int32, numpy=2498>

In [79]:
import tensorflow_probability as tfp

In [80]:
# Find the variance of tensor
tfp.stats.variance(E)

<tf.Tensor: shape=(), dtype=int32, numpy=851>

In [81]:
tf.math.reduce_variance(tf.cast(E, dtype = tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=850.7584>

In [82]:
# find the standard deviation (std)
tf.math.reduce_std(tf.cast(E, dtype = tf.float32)) # we're changing it to float32 bcoz for standardization we need it to be real or complex type

<tf.Tensor: shape=(), dtype=float32, numpy=29.167763>

 find the positional maximum and minimum

 you will use this more on neural network o/p's prediction probobality

In [83]:
# create a new tensor for finding the positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape = [50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [84]:
# find the positional maximum
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [85]:
# Index on our largest value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [86]:
# Find max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [87]:
# check for equality
assert F[tf.argmax(F)] == tf.reduce_max(F) # we use assert bcoz it will show error if they don't equal

In [88]:
# so we get no error while we checking equality 
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [89]:
# finding the positional minimum
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [90]:
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

In [91]:
tf.reduce_min(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

In [92]:
 assert F[tf.argmin(F)] == tf.reduce_min(F) # no error means both are equal

In [93]:
print(tf.__version__)

2.11.0


# Squeezing a tensor

In [94]:
# squezzing a tensor (removing all single dimensions)

# create a tensor
tf.random.set_seed(42)
g = tf.constant(tf.random.uniform(shape = [50]), shape = (1, 1, 1, 1, 50))

g

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [95]:
g.shape

TensorShape([1, 1, 1, 1, 50])

In [96]:
g_sqeezed = tf.squeeze(g)

g_sqeezed

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

One-hot encoding tensors

In [97]:
# one-hot encoding is a form of numerical encoding

# create a list of indices

some_list = [0, 1, 2, 3] # could be red, green, blue, purple

#one-hot encode the list of indices
tf.one_hot(some_list, depth = 4) # we have 4 indices in some_list so we use depth = 4 so it will be 4 by 4

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [98]:
# specify custom values for one-hot encoding
tf.one_hot(some_list, depth = 4, on_value = "I love deep learning", off_value = "I also like to nlp")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'I love deep learning', b'I also like to nlp',
        b'I also like to nlp', b'I also like to nlp'],
       [b'I also like to nlp', b'I love deep learning',
        b'I also like to nlp', b'I also like to nlp'],
       [b'I also like to nlp', b'I also like to nlp',
        b'I love deep learning', b'I also like to nlp'],
       [b'I also like to nlp', b'I also like to nlp',
        b'I also like to nlp', b'I love deep learning']], dtype=object)>

In [99]:
  # what if we give different depths

z = [1, 2, 3, 4]
tf.one_hot(z, depth = 3)

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

In [100]:
tf.one_hot(z, depth = 5)

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)>

In [101]:
i = [[0, 2], [1, -1]]
tf.one_hot(i, depth = 3)

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[1., 0., 0.],
        [0., 0., 1.]],

       [[0., 1., 0.],
        [0., 0., 0.]]], dtype=float32)>

SQUARING, LOG, SQUARE ROOT

In [102]:
# create a new tensor
h = tf.range(1, 10)
h

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [103]:
# square it 
tf.square(h)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81])>

In [104]:
# square root # this method requires non-int types

# tf.math.sqrt(h) # here we will get error bcoz it is in dtype = int32
# we need dtypes like bfloat16, half, float, double, complex64, complex128 

In [105]:
tf.math.sqrt(tf.cast(h, dtype = tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [106]:
# find the log # for this method we need non-int types

#tf.math.log(h) # here we will get error bcoz it is in dtype = int32
# we need dtypes like bfloat16, half, float, double, complex64, complex128 

In [107]:
tf.math.log(tf.cast(h, dtype = tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

Tensors and numpy

Tensorflow interact beautifully with numpy arrays

In [108]:
# create a tensor directly from numpy array
j = tf.constant(np.array([3., 7., 10.]))
j

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [109]:
# convert our tensor back to numpy array
np.array(j), type(np.array(j))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [110]:
# conert tensor j to a numpy array
j.numpy(), type(j.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [111]:
j = tf.constant([3.])
j.numpy()[0]

3.0

In [113]:
# The default types of each are slightly different
numpy_j = tf.constant(np.array([3., 7., 10.]))
tensor_j = tf.constant([3., 7., 10.])
numpy_j.dtype, tensor_j.dtype

(tf.float64, tf.float32)